In [1]:
import os

In [2]:
%pwd

'f:\\ml projects\\chicken_disease_classification\\notebooks'

#### Currently we are in Notebook folder so we have go back in chicken_disease_classification

In [3]:
os.chdir('../')
%pwd

'f:\\ml projects\\chicken_disease_classification'

#### 👇 👇 👇 👇 👇 👇 called Entity paste it in entity/config_entity.py

In [4]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir:Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path 


#### 👇 👇 👇 👇 👇 👇 paste it on src/config/configuration.py

In [5]:
from chicken_disease_classifier.constants import *
from chicken_disease_classifier.utils.common import read_yaml,create_directories

In [6]:
class ConfigurationManager:
    def __init__(self,config_filepath = CONFIG_FILE_PATH,params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root]) 

    def get_data_ingestion_config(self) -> DataIngestionConfig:

        config = self.config.data_ingestion

        data_ingestion_config = DataIngestionConfig(
                                    root_dir= config.root_dir,
                                    source_URL= config.source_URL,
                                    local_data_file= config.local_data_file,
                                    unzip_dir= config.unzip_dir)
        return data_ingestion_config
        

#### 👇 👇 👇 👇 👇 👇 paste it on src/components/data_ingestion.py

In [7]:
import os
from urllib import request
import zipfile
from chicken_disease_classifier import logger
from chicken_disease_classifier.utils.common import get_size ## if Gives error then close the file and again run it or push the common.py on github

In [19]:

class DataIngestion:
    def __init__(self,config: DataIngestionConfig):
        
        self.config = config
        os.makedirs(self.config.root_dir,exist_ok=True)
    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(url= self.config.source_URL, filename= self.config.local_data_file)
            logger.info(f"{filename} download! With Following info:\n {headers}")
        else:
            logger.info(f"File already Exists :{get_size(Path(self.config.local_data_file))}")
            

    def extract_zip_file(self):

        unzip_path = self.config.unzip_dir
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)

#### 👇 👇 👇 👇 👇 👇 paste it in pipeline/stage_01_data_ingestion.py

In [22]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config= data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[ 2023-11-21 15:56:26,928 ] 26 - common - INFO - <_io.TextIOWrapper name='config\\config.yaml' mode='r' encoding='UTF-8'> Loaded Successfully
[ 2023-11-21 15:56:26,931 ] 26 - common - INFO - <_io.TextIOWrapper name='params.yaml' mode='r' encoding='UTF-8'> Loaded Successfully
[ 2023-11-21 15:56:26,933 ] 48 - common - INFO - Created Directory at : artifacts
[ 2023-11-21 15:56:29,526 ] 10 - 4054381443 - INFO - artifacts/data_ingestion/data.zip download! With Following info:
 Connection: close
Content-Length: 11616915
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "57e2d9ae182eda1180abff67b40c8fd40c0b4723c2c465c813da5c28d5f36fbc"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: B452:8E4C:353CAB:40FC8B:655C813C
Accept-Ranges: bytes
Date: Tue, 21 Nov 2023 10:26:27 GMT
Via: 1.1 varnish
X-Served-By: